# Background & clutter

### What it this?

This script imports images, resizes them and pastes them on a background. The latter step is done three ways: using a plain background, a heavily cluttered and a lightly cluttered one.

In addition, the script manipulates cluttered backgrounds so that their pixel value histogram matches that of the average object.

### How does it work?

Very good question. First check the following requirements:
1. Make sure this script is located in a directory (`script`) of which the parent directory contains a folder named `COCOimages` and where all the images you wish to work on are located. Those should be images with a `.png` extension, containing objects on transparent backgrounds.
2. Make sure the parent directory also contains a `bckgrndImages > scrambled_light_clutter` and a `bckgrndImages > scrambled_heavy_clutter` folder, which contain the necessary cluttered background images.

Here are the parameters that need to be set before running the script (check `Preamble`):
- `outSize` defines the final resolution of output images (700x700 by default).
- `obj_imRatio` defines the space the object will take in the final image. More precisely it corresponds to the ratio of the longer side of the object (width/height) to the size of the image (700 by default).
- `col_bckgrnd` defines the color of the plain backgrounds in grayscale value. Leave this value commented out by default: it will take the average pixel value of all objects.

## Preamble

Set the required parameters in the following cell.

In [1]:
# decide on the size of the output image
outSize = (700, 700)

# decide on the size of the object on the image
obj_imRatio = 0.5 # decide on the ratio of the object size relative to image size

# decide on the color of the background (in grayscale 0-255) 
# col_backgrnd = 80

In [2]:
# some packages needed here
import cv2 as cv
import numpy as np
import glob
import os
from PIL import Image
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.exposure import match_histograms
import math

print("All libraries are loaded.")

All libraries are loaded.


In [11]:
# we'll neeed this function as well
def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

### Set the input paths

Input paths are given for input object images _(those are the same for **plain** and **cluttered** backgrounds)_. Inputs paths are also given for cluttered backgrounds.

In [3]:
# give the input path for object images
parentPath = os.pardir
inputPath = parentPath + r'/COCOimages/'

imagePath = glob.glob(inputPath + "*") # take in all the files in there
imagePath.sort() # make sure the files are sorted

# count the number of images to process
nbrIm = len(imagePath)
print('There are', nbrIm, 'images to process.')

There are 53 images to process.


In [4]:
# give the input path for cluttered backgrounds
heavyClutterPath = parentPath + r'/bckgrndImages/scrambled_heavy_clutter/'
lightClutterPath = parentPath + r'/bckgrndImages/scrambled_light_clutter/'

heavyClutterPath = glob.glob(heavyClutterPath + "*") # take in all the files in there
heavyClutterPath.sort()
lightClutterPath = glob.glob(lightClutterPath + "*")
lightClutterPath.sort()

### Set the output paths

Output paths are created for plain backgrounds (`images`) and for cluttered backgrounds (`output/light_clutter` and `output/heavy_clutter`).

In [5]:
# name the output path for plain backgrounds
plain_outputPath = parentPath + r'/images/'

# create the necessary output folder
if not os.path.exists(plain_outputPath):
    os.makedirs(outputPath)

In [6]:
# name the output path for cluttered backgrounds
heavyClutterOutputPath = parentPath + r'/output/clutter/heavy/'
lightClutterOutputPath = parentPath + r'/output/clutter/light/'

listOutputPaths = [heavyClutterOutputPath, lightClutterOutputPath]

# create the necessary output folders
idx = 0

for outputPath in listOutputPaths:
    if not os.path.exists(listOutputPaths[idx]):
            os.makedirs(listOutputPaths[idx])
            idx +=1

---
## Cluttered backgrounds

In this section the backgrounds used are cluttered and not a plain colour. Before pasting objects on their backgrounds, the latter are manipulated so that their histogram matches that of the average of all objects.

### Reisizing images

Here we take our objects and paste them on backgrounds, controlling for their overall size in the middle of the resulting image. This is how it's done:
1. Objects are extracted from their images by drawing a bounding box around them. From this bounding box the longer side is chosen.
2. Objects are pasted at the center of a plain background so that the longer side of their bounding box has a size of `image size * obj_imRatio`.

### Equating background histograms

Backgrounds are manipulated so that the distribution of their pixel values matches that of the average of all objects. Here is how this is done:
   1. For each object, the number of pixels matching **each possible pixel value (0-255)** is counted.
   2. That number is summed up for each pixel value, from which a histogram of **all total values** can be computed.
   3. From that histogram, an image is created that is 1 pixel wide and as long as the total number of pixels of all objects. That single image then has the exact same histogram as that which was derived in `2.`.
   4. All background clutter images are then passed through a histogram matching function that matches their pixel distribution to that of the image created in `3.`.
   
Step `4.` is computed in the next loop, just before objects are being pasted on top of backgrounds.

In [7]:
# calculating the average histogram of all objects

# creating an empty vector of all possible pixel values
pixelValues = np.zeros([256])

# adding the number of pixels contained in every object to the vector
for file in imagePath:

    im = cv.imread(file, -1)
    visible = im[:,:,3] == 255
    b, g, r, a = cv.split(im)
    im = b[visible]

    for pixel in range(len(im)):
        pixel_value = int(im[pixel])
        pixelValues[pixel_value] += 1

In [8]:
# creating the empty array that will hold all the value points of the histogram
ref_array = np.empty(int(np.sum(pixelValues)))

# fill in the empty array
idx = 0

pixels = []
for x in range(256): pixels.append(x)

for i in pixels:
    ref_array[idx : idx + int(pixelValues[i])] = i
    idx += int(pixelValues[i])

# transform the 1d array obtained into a readable image
ref_array = np.reshape(ref_array, (1, len(ref_array)))
ref_im = ref_array.astype('uint8')

# get the average pixel value for plain backgrounds
avg_colour = round(np.mean(ref_array))

In [27]:
# import images, import backgrounds and adjust them, adjust object size, paste objects and export

for i in range(len(imagePath)):

    # get the image
    file = imagePath[i]
    
    # get the backgrounds
    lightBckgrnd = cv.imread(lightClutterPath[i], 0)
    lightBckgrnd = (match_histograms(lightBckgrnd, ref_im)).astype('uint8')
    heavyBckgrnd = cv.imread(heavyClutterPath[i], 0)
    heavyBckgrnd = (match_histograms(heavyBckgrnd, ref_im)).astype('uint8')
    # create an alpha channel
    alphaSquare = np.ones((lightBckgrnd.shape[0], lightBckgrnd.shape[1]), dtype = np.uint8)*255
    lightBckgrnd = cv.merge([lightBckgrnd, lightBckgrnd, lightBckgrnd, alphaSquare])
    heavyBckgrnd = cv.merge([heavyBckgrnd, heavyBckgrnd, heavyBckgrnd, alphaSquare])

    # get the center of the image
    center = outSize[0]//2, outSize[1]//2

    # extract the name of the image
    basename = os.path.basename(imagePath[i])
    name = os.path.splitext(basename)[0]
    
    # remove the number in the name
    for character in name:
        if character.isdigit():
            name = name.replace(character, "")

    # read the image
    im = cv.imread(file, -1)
    mask = ndimage.find_objects(im[:,:,3] > 0)[0] # find the bounding box containing the object
    im = im[mask] # only keep the object

    # find the larger side & its ratio to the smaller one
    if im.shape[0] > im.shape[1]:
        ratioSide = im.shape[0]/im.shape[1]
        height = int(bckgrnd.shape[0] * obj_imRatio)
        width = int(height / ratioSide)
    elif im.shape[0] <= im.shape[1]:
        ratioSide = im.shape[1]/im.shape[0]
        width = int(bckgrnd.shape[1] * obj_imRatio)
        height = int(width / ratioSide)
    
    # create dimensions to resize
    dim = (round_up_to_even(width), round_up_to_even(height))

    # resize image
    resized_im = cv.resize(im, dim, interpolation = cv.INTER_AREA)
    
    # pasting the resized object on each background
    # getting the offset coordinates
    x1, x2 = center[1] - resized_im.shape[1]//2, center[1] + resized_im.shape[1]//2
    y1, y2 = center[0] - resized_im.shape[0]//2, center[0] + resized_im.shape[0]//2
    # amending the alpha values of the image for the loop
    b,g,r,a = cv.split(resized_im)
    a = a / 255.0
    alpha_square = 1.0 - a
    # pasting objects on backgrounds
    for c in range(0, 3):
        lightBckgrnd[y1:y2,x1:x2,c] = (a * resized_im[:, :, c] + alpha_square * lightBckgrnd[y1:y2,x1:x2,c])
        heavyBckgrnd[y1:y2,x1:x2,c] = (a * resized_im[:, :, c] + alpha_square * heavyBckgrnd[y1:y2,x1:x2,c])

    # write the outputs
    cv.imwrite(lightClutterOutputPath + 'lightClutter' + name + str(i) + '.png', lightBckgrnd)
    cv.imwrite(heavyClutterOutputPath + 'heavyClutter' + name + str(i) + '.png', heavyBckgrnd)
    
print('It\'s all done! Go have a coffee now.')

It's all done! Go have a coffee now.


---
## Plain backgrounds

Here we take our objects and paste them on plain backgrounds, controlling for their overall size in the same way as we did before.

By default, the colour of the plain background is equal to the average colour value of all objects.

In [12]:
# import all images, adjust size, paste on background, export them

for i in range(len(imagePath)):

    # extract image
    file = imagePath[i]
    
    # define background colour
    try: col_backgrnd
    except NameError: col_backgrnd = None
    if col_backgrnd is None:
        col_backgrnd = avg_colour # if nothing else stated, background colour is average pixel value
    else:
        col_backgrnd = col_backgrnd
    
    # create a background
    plainSquare = np.ones((outSize[0], outSize[1]), dtype = np.uint8) * col_backgrnd
    alphaSquare = np.ones((outSize[0], outSize[1]), dtype = np.uint8) * 255
    bckgrnd = cv.merge([plainSquare, plainSquare, plainSquare, alphaSquare])

    # extract the name of the image
    basename = os.path.basename(imagePath[i])
    name = os.path.splitext(basename)[0]
    
    # remove the number in the name
    for character in name:
        if character.isdigit():
            name = name.replace(character, "")

    # read the image
    im = cv.imread(file, -1)
    mask = ndimage.find_objects(im[:,:,3] > 0)[0] # find the bounding box containing the object
    im = im[mask] # only keep the object

    # find the larger side & its ratio to the smaller one
    if im.shape[0] > im.shape[1]:
        ratioSide = im.shape[0]/im.shape[1]
        height = int(bckgrnd.shape[0] * obj_imRatio)
        width = int(height / ratioSide)
    elif im.shape[0] <= im.shape[1]:
        ratioSide = im.shape[1]/im.shape[0]
        width = int(bckgrnd.shape[1] * obj_imRatio)
        height = int(width / ratioSide)
    
    # create dimensions to resize
    dim = (round_up_to_even(width), round_up_to_even(height))

    # resize image
    resized_im = cv.resize(im, dim, interpolation = cv.INTER_AREA)

    # pasting the resized object on each background
    # getting the offset coordinates
    center = bckgrnd.shape[0]//2, bckgrnd.shape[1]//2
    x1, x2 = center[1] - resized_im.shape[1]//2, center[1] + resized_im.shape[1]//2
    y1, y2 = center[0] - resized_im.shape[0]//2, center[0] + resized_im.shape[0]//2

    # amending the alpha values of the image for the loop
    b,g,r,a = cv.split(resized_im)
    a = a / 255.0
    alpha_square = 1.0 - a
    # pasting objects on backgrounds
    for c in range(0, 3):
        bckgrnd[y1:y2,x1:x2,c] = (a * resized_im[:, :, c] + alpha_square * bckgrnd[y1:y2,x1:x2,c])
    
    # write the output
    cv.imwrite(plain_outputPath + name + str(i) + '.png', bckgrnd)

print('It\'s all done')

It's all done


_References_

https://stats.stackexchange.com/questions/46429/transform-data-to-desired-mean-and-standard-deviation

Matching luminance histograms:
- https://github.com/aliprf/CV-HistogramMatching

_(some references are lacking here)_